In [134]:
import numpy as np
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV

from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB

import seaborn as sns

In [100]:
exam_info_df = pd.read_csv('data/raw/365_exam_info.csv')
quiz_info_df = pd.read_csv('data/raw/365_quiz_info.csv')
student_engagement_df = pd.read_csv('data/raw/365_student_engagement.csv')
student_hub_questions_df = pd.read_csv('data/raw/365_student_hub_questions.csv')
student_info_df = pd.read_csv('data/raw/365_student_info.csv')
student_learning_df = pd.read_csv('data/raw/365_student_learning.csv')
student_purchases_df = pd.read_csv('data/raw/365_student_purchases.csv')

In [101]:
student_engagement_df.date_engaged = pd.to_datetime(student_engagement_df.date_engaged)
student_hub_questions_df.date_question_asked = pd.to_datetime(student_hub_questions_df.date_question_asked)
student_info_df.date_registered = pd.to_datetime(student_info_df.date_registered)
student_learning_df.date_watched = pd.to_datetime(student_learning_df.date_watched)
student_purchases_df.date_purchased = pd.to_datetime(student_purchases_df.date_purchased)

In [102]:
# def has_student_engaged_with_quizzes(student_id: str):
#     return (student_engagement_df[student_engagement_df.student_id == student_id].engagement_quizzes == 1).any()

# def has_student_engaged_with_exams(student_id: str):
#     return (student_engagement_df[student_engagement_df.student_id == student_id].engagement_exams == 1).any()

In [103]:
# df = student_info_df.copy()

# sum_minutes_watched = pd.merge(student_info_df.student_id, student_learning_df[['student_id', 'minutes_watched']], on='student_id', how='left')
# sum_minutes_watched = sum_minutes_watched.fillna(0)
# sum_minutes_watched = sum_minutes_watched.groupby('student_id').sum().reset_index()

# days_engaged = student_engagement_df[['student_id', 'date_engaged']].groupby('student_id').size().reset_index()
# days_engaged = pd.merge(student_info_df.student_id, days_engaged, on='student_id', how='left').fillna(0).astype(np.int32)
# days_engaged = days_engaged.rename(columns={0: 'days_engaged'})

# df = pd.merge(df.student_id, sum_minutes_watched, on='student_id')
# df = pd.merge(df, days_engaged, on='student_id', how='left')

# df['engaged_with_quizzes'] = df.student_id.map(has_student_engaged_with_quizzes)
# df['engaged_with_exams'] = df.student_id.map(has_student_engaged_with_exams)
# df['engaged_with_qa'] = df.student_id.isin(student_hub_questions_df.student_id)
# df['subscribed'] = df.student_id.isin(student_purchases_df.student_id)

# df = df[~((df.subscribed == True) & (df.days_engaged == 0))]
# df = df.drop('student_id', axis=1)

In [105]:
# df.to_csv('data/processed/ml_dataset.csv', index=False)

In [106]:
df = pd.read_csv('data/processed/ml_dataset.csv')
df

,minutes_watched,days_engaged,engaged_with_quizzes,engaged_with_exams,engaged_with_qa,subscribed
0,0.3,1,False,False,False,False
1,0.0,0,False,False,False,False
2,531.2,29,True,True,False,True
3,0.0,0,False,False,False,False
4,0.0,0,False,False,False,False
...,...,...,...,...,...,...
35118,7.8,1,True,False,False,False
35119,10.4,1,True,False,False,False
35120,0.1,1,False,False,False,False
35121,0.0,0,False,False,False,False


In [136]:
X = df.drop('subscribed', axis=1)
y = df.subscribed

In [137]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5)

In [138]:
X_train.shape[0], X_val.shape[0], X_test.shape[0]

(21073, 7025, 7025)

In [139]:
ros = RandomOverSampler()
rus = RandomUnderSampler()

scaler = StandardScaler()

X_train_oversampled, y_train_oversampled = ros.fit_resample(X_train, y_train)
X_train_resampled, y_train_resampled = rus.fit_resample(X_train_oversampled, y_train_oversampled)

X_train_scaled = scaler.fit_transform(X_train_resampled)

In [140]:
y_train_resampled.value_counts()

subscribed
False    19852
True     19852
Name: count, dtype: int64

In [144]:
X_train_scaled.shape

(39704, 5)